In [8]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [22]:
diabetes = pd.read_csv('pima-indians-diabetes.csv')
#diabetes.head()
name_list = list(diabetes.columns)

cols_to_norm = [name_list[i] for i in range(0, len(name_list) - 3)]
#Normalize
diabetes[cols_to_norm] = diabetes[cols_to_norm].apply(lambda x: (x - x.min())/(x.max() - x.min()))


# Continuous Features
feat_cols = [tf.feature_column.numeric_column(name) for name in cols_to_norm]
age = tf.feature_column.numeric_column('Age')

# Categoriacal Features
assigned_group = tf.feature_column.categorical_column_with_vocabulary_list('Group', ['A', 'B', 'C', 'D'])

#Converting Continuous to Categrical 
age_buckets = tf.feature_column.bucketized_column(age, boundaries=[20, 30, 40, 50, 60, 70])
feat_cols.append(age_buckets)
feat_cols.append(assigned_group)

x_data = diabetes.drop(labels='Class', axis=1)
labels = diabetes['Class']

x_train, x_test, y_train, y_test = train_test_split(x_data, labels, test_size=0.33, random_state=101)
input_func = tf.estimator.inputs.pandas_input_fn(x=x_train, y=y_train, batch_size=10, num_epochs=1000,shuffle=True)
model = tf.estimator.LinearClassifier(feature_columns=feat_cols, n_classes=2)
model.train(input_fn=input_func, steps=1000)

test_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test,y=y_test,batch_size=10,num_epochs=1,shuffle=False)

result = model.evaluate(input_fn=test_input_func)

pred_input_func = tf.estimator.inputs.pandas_input_fn(x=x_test, batch_size=10,num_epochs=1,shuffle=False)

prediction = model.predict(input_fn=pred_input_func)


INFO:tensorflow:Starting evaluation at 2018-08-10-03:04:52
INFO:tensorflow:Restoring parameters from /var/folders/r1/d60hgx6x2g38_9q1m11fp4t40000gn/T/tmp1757u417/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-08-10-03:04:53
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.712598, accuracy_baseline = 0.65748, auc = 0.787356, auc_precision_recall = 0.627942, average_loss = 0.535919, global_step = 1000, label/mean = 0.34252, loss = 5.23552, prediction/mean = 0.394049
INFO:tensorflow:Restoring parameters from /var/folders/r1/d60hgx6x2g38_9q1m11fp4t40000gn/T/tmp1757u417/model.ckpt-1000


[{'class_ids': array([1]),
  'classes': array([b'1'], dtype=object),
  'logistic': array([ 0.55995244], dtype=float32),
  'logits': array([ 0.24096906], dtype=float32),
  'probabilities': array([ 0.44004753,  0.55995244], dtype=float32)},
 {'class_ids': array([1]),
  'classes': array([b'1'], dtype=object),
  'logistic': array([ 0.68127692], dtype=float32),
  'logits': array([ 0.75964618], dtype=float32),
  'probabilities': array([ 0.31872311,  0.68127692], dtype=float32)},
 {'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.40252885], dtype=float32),
  'logits': array([-0.39493918], dtype=float32),
  'probabilities': array([ 0.59747112,  0.40252885], dtype=float32)},
 {'class_ids': array([0]),
  'classes': array([b'0'], dtype=object),
  'logistic': array([ 0.34779868], dtype=float32),
  'logits': array([-0.62872946], dtype=float32),
  'probabilities': array([ 0.65220129,  0.34779868], dtype=float32)},
 {'class_ids': array([0]),
  'classes': arra

In [25]:
### DNN Classifer
dnn_model = tf.estimator.DNNClassifier(hidden_units=[10, 10, 10], feature_columns=feat_cols, n_classes=2)
embedded_group_column = tf.feature_column.embedding_column(assigned_group, dimension=4)
feat_cols.remove(assigned_group)
feat_cols.append(embedded_group_column)

dnn_model.train(input_fn=input_func, steps=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/r1/d60hgx6x2g38_9q1m11fp4t40000gn/T/tmpp7ots0__', '_log_step_count_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/r1/d60hgx6x2g38_9q1m11fp4t40000gn/T/tmpp7ots0__/model.ckpt.
INFO:tensorflow:step = 1, loss = 10.6169
INFO:tensorflow:global_step/sec: 426.749
INFO:tensorflow:step = 101, loss = 6.93071 (0.236 sec)
INFO:tensorflow:global_step/sec: 456.675
INFO:tensorflow:step = 201, loss = 5.76173 (0.219 sec)
INFO:tensorflow:global_step/sec: 464.894
INFO:tensorflow:step = 301, loss = 4.49628 (0.215 sec)
INFO:tensorflow:global_step/sec: 449.194
INFO:tensorflow:step = 401, loss = 5.99285 (0.223 sec)
INFO:tensorflow:global_step/